In [45]:
import numbers
from datetime import timedelta
from datetime import datetime


class TimeZone:
    def __init__(self, name, offset_hours, offset_minutes):
        if name is None or len(str(name).strip()) == 0:
            raise ValueError("Timezone name cannot be empty.")

        self._name = str(name).strip()

        if not isinstance(offset_hours, numbers.Integral):
            raise ValueError("Hour offset must be integer.")

        if not isinstance(offset_minutes, numbers.Integral):
            raise ValueError("Minutes offset must be integer.")

        if abs(offset_hours) > 59:
            raise ValueError("Minutes offset must be between -59 and 59 (inclusive)")

        offset = timedelta(hours=offset_hours, minutes=offset_minutes)
        if offset < timedelta(hours=-12, minutes=0) or offset > timedelta(
            hours=14, minutes=0
        ):
            raise ValueError("Offset must be between -12:00 and +14:00")

        self._offset_hours = offset_hours
        self._offset_minutes = offset_minutes
        self._offset = offset

    @property
    def offset(self):
        return self._offset

    @property
    def name(self):
        return self._name

    def __repr__(self):
        return (
            f"TimeZone(name={self.name}, "
            f"offset_hours={self._offset_hours},"
            f"offset_minutes={self._offset_minutes})"
        )

In [46]:
tz1 = TimeZone("ABC", -2, -15)

In [47]:
class Account:

    IR = 0.05  # class level property
    transaction_id = 100

    def __init__(self, account_num: int, first_name: str, last_name: str, time_zone=TimeZone('UTC',0,0), start_balance=0) -> None:
        
        if account_num is None or not isinstance(account_num, numbers.Integral):
            raise ValueError('Account number cannot be empty and must be Integer.')   
        self._account_num = account_num

        self.first_name = first_name
        self.last_name = last_name

        if start_balance<0:
            raise ValueError('Starting balance can\'t be negative.')   
        self._balance = float(start_balance)

        self.time_zone = time_zone

    @property
    def account_num(self):
        return self._account_num
    
    '''Names properties: First, Last and Full Names'''

    @staticmethod
    def validate_name(value, field_title):
        if value is None or len(value.strip())==0:
           raise ValueError(f'{field_title} can\'t be empty')
        return str(value).strip().capitalize()
    
    @property
    def first_name(self):
        return self._first_name

    @first_name.setter
    def first_name(self, value):
        self._first_name = Account.validate_name(value, 'First name')
    
    @property
    def last_name(self):
        return self._last_name

    @last_name.setter
    def last_name(self, value):
        self._last_name = Account.validate_name(value, 'Last name')


    @property
    def full_name(self):
        return f'{self.first_name} {self.last_name}'

    '''Balance: we can't access it directly'''
    @property
    def balance(self):
        return self._balance
    
    '''TimeZone property'''
    @property
    def time_zone(self):
        return self._time_zone

    @time_zone.setter
    def time_zone(self, value):
        if not isinstance(value,TimeZone):
            raise ValueError('Time Zone must be a valid TimeZone object')
        self._time_zone = value


    @classmethod
    def transaction_update(cls):
        cls.transaction_id+=1

    '''Balance methods'''
    def transaction(self, amount):
        '''Deposit or withdrawal amount and generate confirmation number'''
        if amount>=0:
            transaction_type = "D"
        elif amount<0:
            transaction_type = "W"

        final_balance = self._balance + amount
        if final_balance>=0:
            self._balance = final_balance
        else:
            transaction_type = "X"
            print('You can\'t go negative in your balance')
        
        dt_str = datetime.utcnow().strftime('%Y%m%d%H%M%S')
        Account.transaction_update()
        return f'{transaction_type}-{self.account_num}-{dt_str}-{Account.transaction_id} equals to {amount}. Current balance is {self._balance}'
        

    def apply_IR(self):
        '''Apply interest rate to current balance'''
        self._balance = round(self._balance*(1+self.IR),2)
        dt_str = datetime.utcnow().strftime('%Y%m%d%H%M%S')
        Account.transaction_update()
        return f'I-{self.account_num}-{dt_str}-{Account.transaction_id}. Current balance is {self._balance}'
    

    @staticmethod
    def parse_confirmation_code(confirmation_code, preferred_time_zone=None):
        #D-986542-20230813214554-102
        parts = confirmation_code.split('-')
        if len(parts) != 4:
            raise ValueError('Invalid confirmation code')
        
        transaction_code, account_number, raw_dt_utc, transaction_id = parts

        try:
            dt_utc = datetime.strptime(raw_dt_utc, '%Y%m%d%H%M%S')
        except ValueError as ex:
            raise ValueError('Invalid transaction datetime') from ex # from ex allows stack all errors
        
        if preferred_time_zone is None:
            preferred_time_zone=TimeZone('UTC',0,0)

        if not isinstance(preferred_time_zone, TimeZone):
            raise ValueError('Invalid TimeZone specified')

        dt_preferred = dt_utc + preferred_time_zone.offset
        dt_preferred_str=f"{dt_preferred.strftime('%Y-%m-%d %H:%M:%S')} ({preferred_time_zone.name})"

        return (transaction_code, account_number, dt_preferred_str, transaction_id)

In [48]:
a1 = Account(12345, "j", "k", TimeZone("UTC", -1, -1))

In [49]:
a1.apply_IR()

'I-12345-20230813215706-101. Current balance is 0.0'

In [50]:
a2 = Account(986542, "d", "cacad")

In [51]:
a2.transaction(25)

'D-986542-20230813215706-102 equals to 25. Current balance is 25.0'

In [52]:
a2.apply_IR()

'I-986542-20230813215706-103. Current balance is 26.25'

In [53]:
try:
    a2.balance = 200
except AttributeError as ex:
    print(ex)

can't set attribute 'balance'


In [54]:
Account.transaction_id

103

In [55]:
a2.IR

0.05

In [56]:
a2.parse_confirmation_code('D-986542-20230813215706-102')

('D', '986542', '2023-08-13 21:57:06 (UTC)', '102')

In [60]:
try:
    a2.parse_confirmation_code('D-986542-202308132-106')
except ValueError as ex:
    print(ex)

Invalid transaction datetime
